<a href="https://colab.research.google.com/github/sherryzhou0201/work/blob/master/Automation_of_Map_Generation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automation of Map Generation

The python code I wrote here can only be be run if you have the ArcMap software installed. This is just the code example. All the related data were cleaned. Please only run the script on a virtual machine and follow the FISMA (The Federal Information Security Management Act) to protect children's personal information in Florida.

# Map Genration Using ArcPy

In [ ]:
# the following command was run in the python window in ArcMap (Code will be slightly different if is running outside of the ArcMap)
# execfile(r".....the directory you saved your file.py")
# run the first ym first, manually include all the symbology layers in the mxd 
# edit the legend, and save the new master mxd
# note: unclick all the "map connection" option, so the legend will not automatically updated by the content
# loop through all the ym's

import arcpy, os
from shutil import copyfile
from arcpy import env
from arcpy.sa import *
import pandas as pd

In [ ]:
arcpy.env.overwriteOutput=True
arcpy.RefreshActiveView()


# layout legend (legend was manually created first for the mxd file)
mxd=arcpy.mapping.MapDocument("CURRENT")
df=arcpy.mapping.ListDataFrmaes(mxd)[0]

# legend can also be adjusted by using legend.elementHieght... which allow you to change legend when you add more layers or different maps
# here since all the legend is the same across the maps, I manually set them at the begining

# the following are the three layers we created for the map, named as index, provider, and kernel (detail definiation can refer to our annual report)
index_symbology=arcpy.mapping.ListLayers(mxd,'Index Layer',df)[0]
index_symbology.visible=False
provider_symbology=arcpy.mapping.ListLayers(mxd,'Provider Layer',df)[0]
provider_symbology.visible=False
outKernelDensity_FL_symbology=arcpy.mapping.ListLayers(mxd,'Child Residence Density',df)[0]
outKernelDensity_FL_symbology.visible=False


# check out the "spatial" function, we will need this analysis tool to create kernel density map
arcpy.CheckOutExtension("spatial")

# create the geodatabase for the input data
out_gdb=r"G:\Sherry_Data\ArcGIS\input\InputData2019.gdb"

# create the symbology map layer
symbolgoy_ly_selection=r"G:\Sherry_Data\ArcGIS\input\index_symbology2019.lyr"
symbology_ly_provider=r"G:\Sherry_Data\ArcGIS\input\provider_symbology2019.lyr"
symbology_lyr_kernel=r"G:\Sherry_Data\ArcGIS\input\Kernel_Den_symbology2019.lyr"

# set the spatial reference
prj_file=r"G:\Sherry_Data\ArcGIS\input\WGS 1984.prj"


# Generate monthly maps for each coalition in FL from 2017-2018 fiscal year

In [ ]:
# street map was loaded in the master mxd, no need to load it at monthly basis (street map loading requires more time than other maps, so make sure you do not load it when it is not necessary)
# I was thinking to make a toolbox in ArcMap out of this (more flexible for our users), but did not do it due to the approaching deadline. 
# For whoever will take over my job, think about it and probably try to make a tool box out of it , so even the non-programmer in our team can use this feature

# script for the toolbox
# get paramter text command from users
# YM=arcpy.GetParameterAsText(0)

# In our case, I hard code the YM (year,month)
YM="2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01"
coalition_name="ELC_Alachua,ELC_Big_Bend,ELC_Brevard, ELC_Broward,ELC_Druval,ELC_Emerald_Coast, ELC_Escambia,ELC_Flagler_Volusia,ELC_Florida_Gateway" # here I did not list all coalitions, we have 30 ELC

for ym in YM.split(","):

  ####################
  ## provider layer ##
  ####################

  # this is actually our nmiddle layer, we do this first because it is state-wide
  # we do not need to loop coalition through this
  in_Table_pv=r"G:\Sherry_Data\ArcGIS\input\data_preparation\provider_{}.xlsx\provider_()$".format(ym,ym)
  out_name_pv='provider_XY'
  # change the data provider_XY to geodatabase
  arcpy.TableToTable_conversion(in_table_pv,out_gdb,out_name_pv)
  gdb_table_pv=out_gdb+os.sep+out_name_pv # os.sep means\
  arcpy.MakeTableView_management(gdb_table_pv,"provider_XY")

  # make the xy event layer
  arcpy.MakeXYEventLayer_management("provider_XY","X","Y","provider_point",prj_file)
  # convert the temp layer to the permanant layer
  arcpy.FeatureClassToFeatureClass_conversion("provider_point",out_gdb,"provider_point_lyr")
  # apply symbology
  arcpy.ApplySymbologyFromLayer_management('provider_point_lyr',symbology_lyr_provider)

  print("Provider Layer Completed")


  for coalition_name in coalition_name.split(","):

    ############################
    ##child layer preparetion###
    ############################

    in_table_ch=r"G:\Sherry_Data\ArcGIS\input\data_preparation\child_{}_{}.xlsx\Sheet1$".format(ym,coalition_name)
    out_name_ch='child_XY'

    # change the data child_XY to geodatabase
    arcpy.TableToTable_conversion(in_table_ch,out_gdb,out_name_ch)
    gdb_table_ch=out_gdb+os.sep+out_name_ch # os.sep means \
    arcpy.MakeTableView_management(gdb_table_ch,'child_XY')

    # for this layer, we create a point layer first based on XY
    arcpy.MakeXYEventLayer_management('child_XY',"X","Y","child_point",prj_file)

    # convert the temp layer to the permenant layer
    arcpy.FeatureClassToFeatureClass_conversion("child_point",out_gdb,"child_point")

    # please be aware, the variable name changes after you join the data
    arcpy.gp.KernelDensity_sa("Child_point","NONE",r'\\t001-fs02.fed-ad.ufl.edu\T001bUSRDATA\MyDocs\sherryzhou0201\Documents\ArcGIS\Default.gdb\outKernelDensity',"#","#","SQUARE_MAP_UNITS",'DENSITIES','PLANAR')

    # Before we apply symobolgoy, make sure we use "resample" function to make sure the way we resample our data is BILINEAR
    # arcpy.Resample_management("outKernelDensity","out1",'#',"BILINEAR")
    arcpy.ApplySymbologyFromLayer_management('outKernelDensity',symbology_lyr_kernel)

    print("{} {} Kernel Density map completed".format(ym,coalition_name))


    #################################
    # selection/infrastructure layer #
    #################################

    # create the basic map layer 
    arcpy.MakeFeatureLayer_management("G:\Sherry_Data\ArcGIS\input\zipbnd_2012,edited.shp","map")
    # convert the basic zipcode layer to gdb feature class
    arcpy.FeatureClassToFeatureClass_conversion("map",out_gdb,"basic_zip_map")
    # the previous one is ony temporary in the loop, put it into a layer actually shown
    out_name_basic_mnap="basic_zip_map"
    gdb_table_basic_zip=out_gdb+os.sep+out_name_basic_map
    arcpy.MakeFeatureLayer_management(gdb_table_basic_zip,"basic_zip_map")


    in_table_selection_infra=r"G:\Sherry_Data\ArcGIS\input\data_preparation\index_{}_{}.xlsx\Sheet1$".format(ym,coalition_name)
    arcpy.TableToTable_conversion(in_table_selection_infra,out_gdb,'selection_infrastructure')
    # convert it to the gdb format
    out_name_infra="selection_infrastructure"
    gdb_table_infra=out_gdb+os.sep+out_name_infra # os.sep means\
    arcpy.MakeTableView_management(gdb_table_infra,"selection_infrastructure")


    # join the baisc_zip_map with the selection_infrasturcture data
    arcpy.JoinField_management('basic_zip_map','zip_num','selection_infrastructure','Zip','Zip;coalition;Capacity_HQ;Num_kid_HQ;Capacity_LQ;Num_kid_LQ;selection;infrastructure;quadrant')

    # be aware, to apply symbology, we need to make sure taht the data names in the symbology_lyr is exactly the same as the name used in the current layer
    # otherwise, the "apply symbology" function will not work

    arcpy.ApplySymbologyFromLayer_management('basic_zip_map',symbology_lyr_selection)

    print("index layer {} {} completed".format(ym,coalition_name))


    # remove the unnessary layers
    for lyr in arcpy.mapping.ListLayers(mxd,"",df):
      if lyr.name=="child_point":
        arcpy.mapping.RemoveLayer(df,lyr)
      if lyr.name=="provider_point":
        arcpy.mapping.RemoveLayer(df,lyr)
      if lyr.name=="map":
        arcpy.mapping.RemoveLayer(df,lyr)
      if lyr.name=="outKernelDensity":
        moveLayer=lyr
      if lyr.name=="basic_zip_map":
        moveLayer1=lyr
    
    arcpy.mapping.MoverLayer(df,refLayer,moveLayer,"AFTER")


    arcpy.SelecLayerByAttribute_management("basic_zip_map",'NEW_SELECTION',""" "coalition"='{}' """.format(coalition_name))
    df.zoomToSelectedFeatures()


    # clear_selection -- get rid of the highlighted green line for the coalition
    arcpy.SelecLayerByAttribute_management("basic_zip_map","CLEAR_SELECTION")
    save_direct_jpg=r"G:\Sherry_Data\ArcGIS\output\{}_{}.jpg".format(ym,coalition_name)
    arcpy.mapping.ExportToJPEG(mxd,save_direct_jpg,df_export_width=2250,df_export_height-4500,resolution=300)

    
